# Collecting the Neighborhoods in Toronto.

From [here](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), all the Boroughs and Neighborhoods of Toronto are listed. This page can be scraped to collect the information

Importing necessary Libraries

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re

Reading the URL (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) which contains the information required in a tabular structure.

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print("Fetching Data From:\n",url,"\n========================")
source=requests.get(url).text
print("Completed")

Fetching Data From:
 https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 
Completed


Converting the recieved html content into a soup object from "bs4" package for easier extraction of necessary information. Finding the table containing the postal codes and neighborhood of Toronto.

In [3]:
soup=BeautifulSoup(source)
#print(soup.prettify())
table=soup.find("table",class_="wikitable sortable") 
#print(table.prettify())
rows=table.find_all("tr")
#print(rows)

By iterating through all the rows of the table,the content can be accessed and stored asa a list. Creating a DataFrame using the list containing Postal Codes, Borough and Neighborhood of Toronto.

In [4]:
l=[]
for row in rows:
    td=row.find_all("td")
    r=[d.text for d in td]
    l.append(r)
df=pd.DataFrame(l)


As the first row was actually containg the column header with `<th>` tag and not `<td>` an empty element is returned and stored
in  the list. Deleting the first row and naming the cloumns of the dataframe

In [5]:
df.drop(0,inplace=True)  # as the first row contains the header information
df_col=["PostalCode","Borough","Neighborhood"]
df.columns=df_col
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


In the Neighborhood column the information is ending with `\n` this needs to be removed.
All the rows in the `Borough` column containing `Not assigned` must be deleted.
If the `Neighborhood` is `Not assigned` then the Borough is assigned as the neighbourhood itself.

In [6]:
df["Neighborhood"]=df["Neighborhood"].str.replace("[\\\n]","")
df=df[df.Borough!="Not assigned"]
df.loc[df.Neighborhood=="Not assigned",["Neighborhood"]]=df.loc[df.Neighborhood=="Not assigned"].Borough
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Each Postal Code can contain multiple neighborhoods, so combining all the neighborhoods into single line seprated by `,`.
So there is no duplicity in the `PostalCode` i.e there is only one occurance of each postal code in the dataset

In [7]:
cln_df=df.groupby(["PostalCode","Borough"])['Neighborhood'].apply(lambda x:", ".join(x)).reset_index()
cln_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
cln_df.shape
print("There are {} rows in the Dataset.".format(cln_df.shape[0]))

There are 103 rows in the Dataset.


In [9]:
cln_df.to_csv("toronto_scraped.csv",index=False)

Loading the Dataset containing the latitude and longitude information of the postal codes of Toronto.
Combining the two datasets to have a single dataset with all the information

In [19]:
geo_info=pd.read_csv("Geospatial_Coordinates.csv")
merged_df=cln_df.merge(geo_info,left_on="PostalCode",right_on="Postal Code").drop("Postal Code",1)
merged_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
